#  **Gestión de propiedades y reservas de apartamentos en un portafolio**

## **Calendar** 

Muestra la representación detallada de la disponibilidad y el estado de las propiedades (apartamentos) en un determinado período de tiempo. Aquí está una explicación más detallada de cada campo en el bloque "Calendar":

1. __PropertyId (Id de la propiedad)__: Es una identificación única asociada a cada propiedad, que probablemente se refiere a un apartamento específico dentro del portafolio.

2. __CalendarDate (Fecha en el calendario)__: Representa una fecha específica entre el 01/07/2022 y el 31/12/2022.

3. __Blocked (Estado)__:
    * 0 (No reservado): Indica que el apartamento estaba disponible en esa fecha.
    * 1 (Bloqueado): Significa que el apartamento estaba cerrado y no estaba disponible para reservar en esa fecha.
    * 2 (Reservado): Indica que el apartamento estaba ocupado por clientes en esa fecha.

4. __Property_BookingId (Id de reserva de la propiedad)__: Este campo solo está presente si el estado (Blocked) es igual a 2, lo que significa que la propiedad estaba reservada en esa fecha. En este caso, el __Property_BookingId__ sería el identificador único asociado a esa reserva específica.

## **Propiedades (Properties)**
Proporciona información estática sobre las propiedades del portafolio, como su identificación única y la capacidad de personas que pueden alojarse en cada apartamento.

1. __PropertyId (Id de la propiedad)__: Es una identificación única asociada a cada propiedad (apartamento) en el portafolio.

2. __PR_Sleeps (Capacidad para dormir)__: Indica la capacidad del apartamento en personas, es decir, cuántas personas pueden dormir en ese apartamento.

## **Reservas (Bookings)**
Proporciona información detallada sobre las reservas realizadas, incluyendo el costo total, duración, fecha de creación y el canal a través del cual se realizó la reserva.

1. __Property_BookingId (Id de reserva de la propiedad)__: Identificación única asociada a una reserva específica.

2. __PB_TotalPrice (Precio total de la reserva)__: Representa el costo total de la reserva.

3. __PB_NumNights (Número de noches de la reserva)__: Indica cuántas noches está programada la reserva.

4. __PB_BookingCreatedDate (Fecha de creación de la reserva)__: Muestra la fecha en que se creó la reserva.

5. __PB_BookingChannelOriginId (Id del canal de origen de la reserva)__: Identificación del canal a través del cual se realizó la reserva.

In [109]:
# librerias
import pandas as pd
from herramientas import obtener_resumen_valores_unicos

In [110]:
# Especifica la ruta del archivo Excel
archivo_excel = 'data/Data analysis test.xlsx'

# Carga el archivo Excel en un DataFrame de pandas
calendar    = pd.read_excel(archivo_excel, sheet_name= 'Calendar')
Properties  = pd.read_excel(archivo_excel, sheet_name= 'Properties')
Bookings    = pd.read_excel(archivo_excel, sheet_name= 'Bookings')


## Visión general de la ocupación mensual 

* Calcula la ocupación mensual para cada propiedad en el portafolio. Utiliza los datos del bloque "Calendar" para contar las noches bloqueadas, noches disponibles y las noches reservadas en cada mes.
* La ocupación sera igual : 

         Ocupación (%) = (Dias Reservados / Dias Disponibles) * 100 
                       = (Dias Reservados / ((Date_maxina - Date_minima) - Dias Bloqueado)) * 100

Con el fin de abordar esta situación, utilizaremos los datos de 'Calendar'. Al observar la tabla, se nota la ausencia de valores nulos, a excepción de 'Property_BookingId', que puede presentar un valor NaN cuando 'Blocked' es igual a '1' o '0'.

In [111]:
obtener_resumen_valores_unicos(calendar)

,Columna,Tipo de Dato,Valores Únicos,Total Valores,Valores Nulos,Porcentaje Nulos
0,PropertyId,int64,223,38893,0,0.000000
1,CalendarDate,datetime64[ns],184,38893,0,0.000000
2,Blocked,int64,3,38893,0,0.000000
3,Property_BookingId,float64,6160,31168,7725,19.862186


Podemos calcular el porcentaje de ocupación mensual agrupando los datos por mes y utilizando la función pivot en la columna "Blocked". Con este enfoque, logramos obtener la información necesaria para calcular el porcentaje de ocupación.

In [112]:
calendar_MES = calendar[['CalendarDate', 'Blocked' ]].copy()

# Convierte la columna 'CalendarDate' al formato de fecha
calendar_MES['CalendarDate'] = pd.to_datetime(calendar_MES['CalendarDate'])

# Agrega una nueva columna llamada 'MonthYear' que contiene el mes y el año
calendar_MES['MonthYear'] = calendar_MES['CalendarDate'].dt.to_period('M')

caledar_pivot= calendar_MES [['MonthYear', 'Blocked']].copy()

# Convertir el DataFrame original a un formato de tabla pivote
df_pivot = pd.pivot_table(caledar_pivot, index='MonthYear', columns='Blocked', aggfunc=len, fill_value=0)

# Mostrar el DataFrame pivote
df_pivot['suma'] = df_pivot [0]+ df_pivot[1] + df_pivot[2]
df_pivot['ocupacion(%)'] = (df_pivot[2] / (df_pivot['suma'] - df_pivot[1])) * 100
df_pivot

Blocked,0,1,2,suma,ocupacion(%)
MonthYear,,,,,
2022-07,921,63,5268,6252,85.118759
2022-08,916,95,5456,6467,85.624608
2022-09,1129,71,5190,6390,82.133249
2022-10,908,83,5587,6578,86.020015
2022-11,1148,330,4981,6459,81.269375
2022-12,1744,317,4686,6747,72.877138


Se puede apreciar una disminución en el porcentaje de ocupación durante los meses de noviembre y diciembre, lo cual se representa de manera más clara en la siguiente gráfica.

In [113]:
import plotly.express as px

# Del DataFrame tiene 'MonthYear' como índice y se llama df_pivot
df_pivot.reset_index(inplace=True)

# Convertir 'MonthYear' a cadena antes de crear el gráfico
df_pivot['MonthYear'] = df_pivot['MonthYear'].astype(str)

# Redondear los valores de ocupacion(%) al centésimo
df_pivot['ocupacion_rounded'] = df_pivot['ocupacion(%)'].round(2)

# Crear el gráfico de líneas con etiquetas sobre los puntos y tamaño ajustado
fig = px.line(df_pivot, x='MonthYear', y='ocupacion(%)', markers=True, line_shape='linear',
              text=df_pivot['ocupacion_rounded'], title='Ocupacion(%) del portafolio')

fig.update_xaxes(title='MonthYear')
fig.update_yaxes(title='Ocupacion(%)')

# Configurar la posición del texto sobre los puntos y el tamaño del texto
fig.update_traces(textposition='top center', textfont_size=15)

# Ajustar el tamaño del gráfico
fig.update_layout(
    height=500,  # ajustar según sea necesario
    width=650    # ajustar según sea necesario
)

# Muestra el gráfico interactivo
fig.show()


## Ocupacion por propiedad(%)

Los datos comprenden el período desde el 31 de diciembre de 2022 a las 00:00:00 hasta el 1 de julio de 2022 a las 00:00:00, lo que equivale a un total de 184 días.

In [114]:
# Da formato a 'CalendarDate'
calendar['CalendarDate'] = pd.to_datetime(calendar['CalendarDate'])

# Encuentra la fecha máxima y mínima
fecha_maxima = calendar['CalendarDate'].max()
fecha_minima = calendar['CalendarDate'].min()

# Calcula la diferencia en días
diferencia_en_dias = (fecha_maxima - fecha_minima).days
# print(f"Entre({fecha_maxima}) y ({fecha_minima}) hay un total de {diferencia_en_dias + 1} días")

Podemos utilizar **'PropertyId'** y, a través del indicador **'Blocked'**, calcular el número de días que la propiedad ha pasado en cada uno de los tres estados.

In [115]:
# Convertir la columna 'CalendarDate' a objetos de fecha
calendar['CalendarDate'] = pd.to_datetime(calendar['CalendarDate'])

# Resumen según los valores únicos en 'PropertyId'
resumen = calendar.groupby('PropertyId')['Blocked'].value_counts().unstack(fill_value=0)

# Suma de dias
resumen['Suma'] = resumen[0] + resumen[1]+ resumen[2]

# Imprimir el resumen
propiedades = len(resumen)
print(f"Se obtiene un total de {propiedades} propiedades diferentes, junto con la cantidad de días que han pasado en cada estado, así como la 'Suma'" )
resumen.head(5)


Se obtiene un total de 223 propiedades diferentes, junto con la cantidad de días que han pasado en cada estado, así como la 'Suma'


Blocked,0,1,2,Suma
PropertyId,,,,
2883,23,1,160,184
3963,35,1,148,184
3964,17,1,166,184
4138,21,7,156,184
4181,22,1,161,184


Al aplicar un filtro, podemos identificar 29 propiedades con una suma de días que difiere de 184. Esto podría deberse a la falta de registros para estas propiedades o a que ingresaron o quedaron fuera durante el período en el que se recopilaron los datos.

In [116]:
# Filtrar las filas donde 'Suma' es diferente de 184
df_filtrado = resumen.loc[resumen['Suma'] != 184]
df_ordenado = df_filtrado.sort_values(by='Suma', ascending=False)
df_ordenado.head(10)

Blocked,0,1,2,Suma
PropertyId,,,,
43330,44,1,136,181
43332,16,3,154,173
43333,23,1,149,173
43334,19,2,150,171
43313,18,5,143,166
43337,24,2,138,164
43340,31,2,124,157
43338,45,11,97,153
43344,60,7,70,137


También es posible considerar la fecha máxima y la fecha mínima para cada propiedad, y a partir de estas fechas, calcular el número de días disponibles para cada propiedad. Si hay concordancia en los registros, es muy probable que algunas propiedades hayan quedado fuera o ingresado durante el período en el que se recopilaron los datos y que no sea un error en los datos o datos faltantes.

In [117]:
# Convertir la columna 'CalendarDate' al tipo de dato datetime
calendar['CalendarDate'] = pd.to_datetime(calendar['CalendarDate'])

# Agregar la nueva columna con el valor máximo de 'CalendarDate' para cada 'PropertyId'
calendar['MaxCalendarDate'] = calendar.groupby('PropertyId')['CalendarDate'].transform('max')
calendar['MinCalendarDate'] = calendar.groupby('PropertyId')['CalendarDate'].transform('min')

# Restar las columnas y crear una nueva columna 'DiasDiferencia'
calendar['DiasDiferencia'] = (calendar['MaxCalendarDate'] - calendar['MinCalendarDate']).dt.days
calendar['DiasDiferencia'] = calendar['DiasDiferencia'] + 1
calendar[['PropertyId','CalendarDate','CalendarDate','MaxCalendarDate','MinCalendarDate','DiasDiferencia']].head(5)

,PropertyId,CalendarDate,CalendarDate,MaxCalendarDate,MinCalendarDate,DiasDiferencia
0,42997,2022-07-01,2022-07-01,2022-12-31,2022-07-01,184
1,42997,2022-07-02,2022-07-02,2022-12-31,2022-07-01,184
2,42997,2022-07-03,2022-07-03,2022-12-31,2022-07-01,184
3,42997,2022-07-04,2022-07-04,2022-12-31,2022-07-01,184
4,42997,2022-07-05,2022-07-05,2022-12-31,2022-07-01,184


Filtramos los datos para cuando 'DiasDiferencia' es diferente a 184 para comparar con los datos anteriores

In [118]:
# Imprimir el DataFrame resultante
df_filtrado = calendar.loc[calendar['DiasDiferencia'] != 184]
df_filtrado = df_filtrado[['PropertyId', 'DiasDiferencia']]
df_filtrado= df_filtrado.drop_duplicates()
df_ordenado = df_filtrado.sort_values(by='DiasDiferencia', ascending=False)
# LISTA PARA PRUEBAS
# lista_property_id = df_ordenado['PropertyId'].tolist()
df_ordenado.head(10)

,PropertyId,DiasDiferencia
35403,43330,181
35584,43332,173
35757,43333,173
35930,43334,171
34317,43313,166
36101,43337,164
36532,43340,157
36265,43338,153
36876,43344,137
37287,43347,137


Las tablas coinciden; podemos asumir que algunas propiedades quedaron fuera o ingresaron durante el período en el que se recopilaron los datos. Con esta información, podemos calcular el porcentaje de ocupación para cada propiedad tomando en cuenta el tiempo que la propiedad formo parte del portafolio.

            Ocupación (%) = (Dias Reservados / Dias Disponibles) * 100

In [119]:
# calculo de ocupacion
resumen['ocupacion(%)'] = (resumen[2] / (resumen['Suma'] - resumen[1])) * 100
resumen.head(15)

Blocked,0,1,2,Suma,ocupacion(%)
PropertyId,,,,,
2883,23,1,160,184,87.431694
3963,35,1,148,184,80.874317
3964,17,1,166,184,90.710383
4138,21,7,156,184,88.135593
4181,22,1,161,184,87.978142
4259,31,2,151,184,82.967033
4710,12,0,172,184,93.478261
4785,19,1,164,184,89.617486
4786,16,3,165,184,91.160221


Con los datos obtenidos, podemos representar gráficamente las propiedades que tienen el porcentaje de ocupación más alto y más bajo.

In [120]:
# Ordenar el DataFrame por 'ocupacion(%)' de forma descendente y tomar el top 15
resumen = resumen.reset_index()
resumen['PropertyId'] = resumen['PropertyId'].astype('str')

top_10_resumen = resumen.sort_values(by='ocupacion(%)', ascending=False).head(15)

# Crear el gráfico de barras horizontales
fig = px.bar(top_10_resumen, x='ocupacion(%)', y='PropertyId', orientation='h',
             labels={'ocupacion(%)': 'Ocupación (%)', 'PropertyId': 'PropertyId'})

# Agregar etiquetas a las barras
fig.update_traces(text=top_10_resumen['ocupacion(%)'].round(2).astype(str) + '%',
                  textposition='inside',  # Puedes ajustar la posición de las etiquetas
                  showlegend=False)  # Para ocultar las etiquetas en la leyenda

# Personalizar el diseño del gráfico
fig.update_layout(
    title='Top 15 PropertyId por Ocupación',
    xaxis_title='Ocupación (%)',
    yaxis_title='PropertyId',
)

# Ajustar el tamaño del gráfico
fig.update_layout(
    height=500,  # ajustar según sea necesario
    width=650    # ajustar según sea necesario
)

# Mostrar el gráfico
fig.show()

In [121]:
# Ordenar el DataFrame por 'ocupacion(%)' de forma descendente y tomar el top 15
top_10_resumen = resumen.sort_values(by='ocupacion(%)', ascending=True).head(15)

# Crear el gráfico de barras horizontales
fig = px.bar(top_10_resumen, x='ocupacion(%)', y='PropertyId', orientation='h',
             labels={'ocupacion(%)': 'Ocupación (%)', 'PropertyId': 'PropertyId'})

# Agregar etiquetas a las barras
fig.update_traces(text=top_10_resumen['ocupacion(%)'].round(2).astype(str) + '%',
                  textposition='inside',  # Puedes ajustar la posición de las etiquetas
                  showlegend=False)  # Para ocultar las etiquetas en la leyenda

# Personalizar el diseño del gráfico
fig.update_layout(
    title='Bottom 15 PropertyId por Ocupación',
    xaxis_title='Ocupación (%)',
    yaxis_title='PropertyId',
)

# Ajustar el tamaño del gráfico
fig.update_layout(
    height=500,  # ajustar según sea necesario
    width=650    # ajustar según sea necesario
)

# Mostrar el gráfico
fig.show()


La gráfica exhibe **9 propiedades** que no alcanzaron el **50% de ocupación** durante los **6 meses** analizados, de las cuales 7 permanecieron desocupadas en todo ese período. Es necesario investigar las razones detrás de esta baja ocupación.

## ADR (Tarifa Diaria Promedio)

Para calcular el ADR (Tarifa Diaria Promedio), utilizaré los datos de **'Bookings'**, a los cuales agregaremos la variable **'PR_Sleeps'** obtenida de **'Properties'**.

In [122]:
Bookings['ADR'] = Bookings['PB_TotalPrice'] / Bookings['PB_NumNights']

In [123]:
# Creacion de una lista de valores unicos 'Property_BookingId'
property_booking_ids_uniques = Bookings['Property_BookingId'].unique()

# Filtrar el DataFrame 'calendar' para incluir solo las filas donde el 'Property_BookingId'
# está presente en la lista 'property_booking_ids_uniques'
calendar_filtrado = calendar[calendar['Property_BookingId'].isin(property_booking_ids_uniques)]

# Muestra el DataFrame filtrado
calendar_filtrado = calendar_filtrado[['PropertyId', 'Property_BookingId']].copy()
# Elimina duplicados
calendar_filtrado_sin_duplicados = calendar_filtrado.drop_duplicates().copy()

# Convertir la columna 'Property_BookingId' en el DataFrame 'calendar_filtrado_sin_duplicados'
# a tipo de dato entero de 64 bits (int64) para asegurar que sea de tipo numérico.
calendar_filtrado_sin_duplicados['Property_BookingId'] = calendar_filtrado_sin_duplicados['Property_BookingId'].astype('int64')


In [124]:
# Fusionar (merge) los DataFrames 'Bookings' y 'calendar_filtrado_sin_duplicados' utilizando la columna 'Property_BookingId'
# como clave de unión, de manera interna (inner join), para obtener el DataFrame 'resultado_union'.
resultado_union = pd.merge(Bookings, calendar_filtrado_sin_duplicados, on='Property_BookingId', how='inner')

# Muestra el resultado de la unión
# resultado_union


In [125]:
# Fusionar (merge) el DataFrame 'resultado_union' con el DataFrame 'Properties' utilizando la columna 'PropertyId'
# como clave de unión, de manera interna (inner join), para obtener el DataFrame 'resultado_union_2'.
resultado_union_2 = pd.merge(resultado_union, Properties, on='PropertyId', how='inner')

# Muestra el resultado de la unión
resultado_union_2.head(5)


,Property_BookingId,PB_TotalPrice,PB_NumNights,PB_BookingCreatedDate,PB_BookingChannelOriginId,ADR,PropertyId,PR_Sleeps
0,103656,492.71,2,2022-06-28 20:10:07,7,246.355000,42997,4
1,102879,2130.51,11,2022-06-13 14:27:03,7,193.682727,42997,4
2,104627,348.10,2,2022-07-13 18:44:25,1,174.050000,42997,4
3,104725,659.71,4,2022-07-15 09:59:53,7,164.927500,42997,4
4,103466,618.51,3,2022-06-25 15:43:49,7,206.170000,42997,4


Resumen de los datos obtenidos 

In [126]:
obtener_resumen_valores_unicos(resultado_union_2)

,Columna,Tipo de Dato,Valores Únicos,Total Valores,Valores Nulos,Porcentaje Nulos
0,Property_BookingId,int64,6160,6160,0,0.0
1,PB_TotalPrice,float64,5862,6160,0,0.0
2,PB_NumNights,int64,63,6160,0,0.0
3,PB_BookingCreatedDate,datetime64[ns],6159,6160,0,0.0
4,PB_BookingChannelOriginId,int64,13,6160,0,0.0
5,ADR,float64,5994,6160,0,0.0
6,PropertyId,int64,216,6160,0,0.0
7,PR_Sleeps,int64,9,6160,0,0.0


Es posible calcular el **ADR** (Tarifa Diaria Promedio) según el canal agrupando los datos en base a dicho canal y calculando el promedio. Esto nos permitirá obtener una visión segmentada del desempeño del **ADR** en función de los diferentes canales utilizados.

In [127]:
# Agrupar el DataFrame 'resultado_union_2' por la columna 'PB_BookingChannelOriginId'
# para crear un objeto de grupo llamado 'grupo_canal'.
grupo_canal = resultado_union_2.groupby('PB_BookingChannelOriginId')

# Calcular el valor promedio de la columna 'ADR' para cada grupo en 'grupo_canal'
# y almacenar los resultados en 'adr_promedio_por_canal'.
adr_promedio_por_canal = grupo_canal['ADR'].mean()

# Convierte la serie resultante en un DataFrame
df_adr_promedio_por_canal = adr_promedio_por_canal.reset_index()

# Muestra y ordena el DataFrame resultante
df_adr_promedio_por_canal = df_adr_promedio_por_canal.sort_values(by='ADR', ascending=False)
df_adr_promedio_por_canal


,PB_BookingChannelOriginId,ADR
3,9,2772.568532
8,32,338.859123
7,30,328.518720
11,39,309.778073
10,37,235.676954
0,1,233.339862
2,7,222.429914
5,16,222.324951
4,14,192.617544
12,44,141.787248


El ADR del **canal 9** sobresale notablemente, como se evidencia en la siguiente gráfica. Es necesario verificar la precisión de los datos y determinar si este resultado excepcional se debe a la exactitud de la información o a algún error en la recopilación de datos.

In [128]:
import plotly.express as px

# PB_BookingChannelOriginId a str
df_adr_promedio_por_canal['PB_BookingChannelOriginId'] = df_adr_promedio_por_canal['PB_BookingChannelOriginId'].astype('str')

fig = px.bar(df_adr_promedio_por_canal, x='ADR', y='PB_BookingChannelOriginId', orientation='h', title='ADR Promedio por Canal de Reserva')
fig.update_layout(xaxis_title='ADR Promedio', yaxis_title='PB_BookingChannelOriginId')

# Ajustar el tamaño del gráfico
fig.update_layout(
    height=500,  # ajustar según sea necesario
    width=650    # ajustar según sea necesario
)

# Muestra el gráfico interactivo
fig.show()


Los datos indican un alquiler de una propiedad por un monto de **1,809,468.00**, lo cual no se corresponde con las demás cifras registradas para el alquiler de esa misma propiedad.

In [129]:
# Filtra 'PB_BookingChannelOriginId' igual a 9
resultado_filtrado = resultado_union_2[resultado_union_2['PB_BookingChannelOriginId'] == 9]

# Muestra el DataFrame filtrado
# Muestra el DataFrame resultante
resultado_filtrado = resultado_filtrado.sort_values(by='ADR', ascending=False)
resultado_filtrado[['Property_BookingId','PropertyId','PB_TotalPrice','PB_NumNights','PB_BookingCreatedDate']].head(5)


,Property_BookingId,PropertyId,PB_TotalPrice,PB_NumNights,PB_BookingCreatedDate
1585,108110,42684,1809468.00,6,2022-09-15 19:55:33
4194,104425,43168,1144.63,1,2022-07-10 18:34:59
4895,103373,43230,2890.76,4,2022-06-23 16:40:37
1755,104261,42892,3559.61,6,2022-07-07 16:02:47
5480,108570,43302,562.88,1,2022-09-28 15:09:56


Es necesario eliminar este dato ya que parece ser un error, ya que el monto de alquiler de **1,809,468.00** no concuerda con las demás cifras registradas para el alquiler de esa misma propiedad.

In [130]:
# Filtra PropertyId 
resultado_filtrado = resultado_union_2[resultado_union_2['PropertyId'] == 42684]

# Muestra el DataFrame filtrado
# Muestra el DataFrame resultante
resultado_filtrado = resultado_filtrado.sort_values(by='ADR', ascending=False)
resultado_filtrado[['Property_BookingId','PropertyId','PB_TotalPrice','PB_NumNights','PB_BookingCreatedDate']].head(5)

,Property_BookingId,PropertyId,PB_TotalPrice,PB_NumNights,PB_BookingCreatedDate
1585,108110,42684,1809468.00,6,2022-09-15 19:55:33
1564,102219,42684,1806.50,4,2022-05-31 21:23:57
1575,105824,42684,1559.26,4,2022-08-04 04:27:42
1572,103338,42684,1890.00,5,2022-06-22 22:11:53
1569,104774,42684,1105.26,3,2022-07-15 22:31:26


Una vez eliminados los datos incorrectos, podemos representar gráficamente el ADR en función del canal.

In [131]:
# Filtro de el DataFrame resultado_union_2
resultado_filtrado = resultado_union_2[resultado_union_2['ADR'] != 301578]

# Muestra el DataFrame filtrado
# resultado_filtrado.head(5)


In [132]:
# Filtro de el DataFrame resultado_filtrado
grupo_canal = resultado_filtrado.groupby('PB_BookingChannelOriginId')

# Ahora, puedes aplicar funciones agregadas a cada grupo, por ejemplo, calcular el promedio del ADR
adr_promedio_por_canal = grupo_canal['ADR'].mean()

# Convierte la serie resultante en un DataFrame 
df_adr_promedio_por_canal = adr_promedio_por_canal.reset_index()

# Muestra el DataFrame resultante
df_adr_promedio_por_canal = df_adr_promedio_por_canal.sort_values(by='ADR', ascending=False)
df_adr_promedio_por_canal


# PB_BookingChannelOriginId a str
df_adr_promedio_por_canal['PB_BookingChannelOriginId'] = df_adr_promedio_por_canal['PB_BookingChannelOriginId'].astype('str')

fig = px.bar(df_adr_promedio_por_canal, x='ADR', y='PB_BookingChannelOriginId', orientation='h', title='ADR Promedio por Canal de Reserva')
fig.update_layout(xaxis_title='ADR Promedio', yaxis_title='Canal de reserva')

# Agrega etiquetas en las barras
fig.update_traces(text=df_adr_promedio_por_canal['ADR'].round(2).astype(str),
                  textposition='inside',  # Puedes ajustar esto según tus preferencias
                  textfont=dict(color='white'))  # Puedes ajustar el color del texto

# Ajustar el tamaño del gráfico
fig.update_layout(
    height=500,  # ajustar según sea necesario
    width=650    # ajustar según sea necesario
)

# Muestra el gráfico interactivo
fig.show()


Se observa que los canales **(32, 30, 39 y 9)** tienen el ADR más alto, mientras que los canales **(36, 28, 5 y 44)** muestran un ADR significativamente más bajo.

También podemos visualizar gráficamente el **ADR** en relación con la capacidad de personas que pueden hospedarse en la propiedad.

In [133]:
# Suponiendo que tienes el DataFrame resultado_union_2
grupo_canal = resultado_filtrado.groupby('PR_Sleeps')

# Ahora, puedes aplicar funciones agregadas a cada grupo, por ejemplo, calcular el promedio del ADR
adr_promedio_por_canal = grupo_canal['ADR'].mean()

# Convierte la serie resultante en un DataFrame si es necesario
df_adr_promedio_por_canal = adr_promedio_por_canal.reset_index()

# Muestra el DataFrame resultante
df_adr_promedio_por_canal = df_adr_promedio_por_canal.sort_values(by='ADR', ascending=False)
df_adr_promedio_por_canal


# PR_Sleeps a str
df_adr_promedio_por_canal['PR_Sleeps'] = df_adr_promedio_por_canal['PR_Sleeps'].astype('str')

fig = px.bar(df_adr_promedio_por_canal, x='ADR', y='PR_Sleeps', orientation='h', title='ADR Promedio por Capacidad del apartamento en personas ')
fig.update_layout(xaxis_title='ADR Promedio', yaxis_title='Capacidad del apartamento en personas')

# Agrega etiquetas en las barras
fig.update_traces(text=df_adr_promedio_por_canal['ADR'].round(2).astype(str),
                  textposition='inside',  # Puedes ajustar esto según tus preferencias
                  textfont=dict(color='white'))  # Puedes ajustar el color del texto

# Ajustar el tamaño del gráfico
fig.update_layout(
    height=500,  # ajustar según sea necesario
    width=650    # ajustar según sea necesario
)

# Muestra el gráfico interactivo
fig.show()

In [134]:
# filtro
resultado_filtrado = resultado_union_2[resultado_union_2['PR_Sleeps'] == 1]

# Muestra el DataFrame filtrado
# Muestra el DataFrame resultante
#resultado_filtrado = resultado_filtrado.sort_values(by='ADR', ascending=False)
#resultado_filtrado.head(20)

## Comparación del porcentaje de ocupación mes a mes.

Desde **'calendario'**, extraeremos la fecha **'CalendarDate'** y la información correspondiente al estado de la propiedad **'Blocked'**. Estos datos se agruparán por día para calcular el porcentaje de ocupación. Una vez obtenidos estos datos, procederemos a representar gráficamente la evolución del porcentaje de ocupación mes a mes.

In [135]:
calendar_MES = calendar[['CalendarDate', 'Blocked' ]].copy()

# Convierte la columna 'CalendarDate' al formato de fecha
calendar_MES['CalendarDate'] = pd.to_datetime(calendar_MES['CalendarDate'])

# copia
caledar_pivot= calendar_MES [['CalendarDate', 'Blocked']].copy()

# Convertir el DataFrame original a un formato de tabla pivote
df_pivot = pd.pivot_table(caledar_pivot, index='CalendarDate', columns='Blocked', aggfunc=len, fill_value=0)

df_pivot['suma'] = df_pivot [0]+ df_pivot[1] + df_pivot[2]

df_pivot['ocupacion(%)'] = (df_pivot[2] / (df_pivot['suma'] - df_pivot[1])) * 100

pivot_df = df_pivot.reset_index()
#pivot_df.head(5)

In [136]:
pivot_df = pivot_df[['CalendarDate', 'ocupacion(%)']].copy()
#pivot_df

In [137]:
import warnings

# Ignorar la advertencia específica relacionada con la agrupación en Pandas
warnings.filterwarnings("ignore", message="When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas.*")


In [138]:
# Agregar una columna 'Day' para representar los días del 1 al 30
pivot_df['Day'] = pivot_df['CalendarDate'].dt.day.copy()

# Crear un gráfico de líneas con un color diferente para cada mes
fig = px.line(pivot_df, x='Day', y='ocupacion(%)', color=pivot_df['CalendarDate'].dt.month_name())

# Personalizar el diseño del gráfico
fig.update_layout(
    title='Ocupación Mensual',
    xaxis_title='Días del Mes',
    yaxis_title='Ocupación (%)',
    legend_title='Mes',
    hovermode='x unified'  # Permite mostrar la información al pasar el ratón sobre el gráfico
)

# Ajustar el tamaño del gráfico
fig.update_layout(
    height=500,  # ajustar según sea necesario
    width=650    # ajustar según sea necesario
)

# Mostrar el gráfico
fig.show()

Se observa una disminución en el porcentaje de ocupación que comienza en noviembre y continúa en descenso hacia diciembre, aunque logra recuperarse a finales de dicho mes.

# Análisis de Reservas

Los datos indican que el **94%** de las reservas se realizan a través de los canales **('7' y '1')**, mientras que los otros **11 canales** representan solo el **6%** de las reservas.
Pdemos ver esto en la siguiente gráfica

In [139]:
# Calcular la frecuencia de los valores en 'PB_BookingChannelOriginId'
frecuencia_valores = Bookings['PB_BookingChannelOriginId'].value_counts()

# Convertir la serie de frecuencias en un DataFrame
df_frecuencia = pd.DataFrame({'Canal': frecuencia_valores.index, 'Numero de reservas': frecuencia_valores.values})

# Calcular el porcentaje y agregar una nueva columna
df_frecuencia['Nº reservas (%)'] = (df_frecuencia['Numero de reservas'] / df_frecuencia['Numero de reservas'].sum()) * 100

# Mostrar el DataFrame resultante
# df_frecuencia

In [140]:

# Crear una nueva columna 'Canal_Agrupado' que combine '7' y '1' en 'Otros'
df_frecuencia['Canal_Agrupado'] = df_frecuencia['Canal'].apply(lambda x: f'Canal {x}' if x in [7, 1] else 'Otros')

# Filtrar el DataFrame para incluir solo las categorías '7', '1' y 'Otros'
df_filtrado = df_frecuencia[df_frecuencia['Canal_Agrupado'].isin(['Canal 7', 'Canal 1', 'Otros'])]

# Crear el gráfico de pie interactivo
fig = px.pie(df_filtrado, values='Nº reservas (%)', names='Canal_Agrupado',
             title='Distribución de Reservas por Canal',
             labels={'Canal_Agrupado': 'Canal'},
             hole=0.2)

# # Simular "pull" mediante anotaciones
# fig.add_annotation(
#     text='Canal 1',  # Etiqueta de la anotación
#     x=0.7,  # Coordenada x
#     y=0.5,  # Coordenada y
#     showarrow=False,  # No mostrar flecha
#     font=dict(size=10)  # Ajustar el tamaño del texto
# )

# Ajustar el tamaño del gráfico
fig.update_layout(
    height=500,  # ajustar según sea necesario
    width=650    # ajustar según sea necesario
)


# Mostrar el gráfico interactivo
fig.show()


In [141]:
# import plotly.graph_objects as go
# 
# # Crear una nueva columna 'Canal_Agrupado' que combine '7' y '1' en 'Otros'
# df_frecuencia['Canal_Agrupado'] = df_frecuencia['Canal'].apply(lambda x: f'Canal {x}' if x in [7, 1] else 'Otros')
# 
# # Filtrar el DataFrame para incluir solo las categorías 'Canal 7', 'Canal 1' y 'Otros'
# df_filtrado = df_frecuencia[df_frecuencia['Canal_Agrupado'].isin(['Canal 7', 'Canal 1', 'Otros'])]
# 
# # Datos para el gráfico de pie
# labels = df_filtrado['Canal_Agrupado']
# values = df_filtrado['Nº reservas (%)']
# 
# # Configurar la funcionalidad de "pull"
# pull_values = [0.1, 0.1, 0.1]
# 
# # Crear el gráfico de pie
# fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=pull_values)])
# 
# # Ajustar el diseño del gráfico
# fig.update_layout(
#     title='Distribución de Reservas por Canal',
#     height=500,
#     width=650
# )
# 
# # Mostrar el gráfico interactivo
# fig.show()
# 

In [146]:
Properties

,PropertyId,PR_Sleeps
0,42997,4
1,42998,4
2,43025,4
3,43119,4
4,43120,4
...,...,...
318,43497,5
319,43263,7
320,43264,6
321,43350,5


In [142]:
Bookings

,Property_BookingId,PB_TotalPrice,PB_NumNights,PB_BookingCreatedDate,PB_BookingChannelOriginId,ADR
0,103656,492.71,2,2022-06-28 20:10:07,7,246.355000
1,102879,2130.51,11,2022-06-13 14:27:03,7,193.682727
2,104627,348.10,2,2022-07-13 18:44:25,1,174.050000
3,104725,659.71,4,2022-07-15 09:59:53,7,164.927500
4,103466,618.51,3,2022-06-25 15:43:49,7,206.170000
...,...,...,...,...,...,...
6155,111975,511.53,4,2022-12-25 10:34:16,7,127.882500
6156,111685,1195.68,4,2022-12-17 13:06:45,7,298.920000
6157,111686,706.32,2,2022-12-17 13:10:53,7,353.160000
6158,108525,2622.54,37,2022-09-27 11:58:56,7,70.879459


In [143]:
calendar

,PropertyId,CalendarDate,Blocked,Property_BookingId,MaxCalendarDate,MinCalendarDate,DiasDiferencia
0,42997,2022-07-01,2,103656.0,2022-12-31,2022-07-01,184
1,42997,2022-07-02,2,103656.0,2022-12-31,2022-07-01,184
2,42997,2022-07-03,0,NaN,2022-12-31,2022-07-01,184
3,42997,2022-07-04,2,102879.0,2022-12-31,2022-07-01,184
4,42997,2022-07-05,2,102879.0,2022-12-31,2022-07-01,184
...,...,...,...,...,...,...,...
38888,43350,2022-12-27,0,NaN,2022-12-31,2022-09-24,99
38889,43350,2022-12-28,0,NaN,2022-12-31,2022-09-24,99
38890,43350,2022-12-29,0,NaN,2022-12-31,2022-09-24,99
38891,43350,2022-12-30,0,NaN,2022-12-31,2022-09-24,99


In [144]:
obtener_resumen_valores_unicos(calendar)

,Columna,Tipo de Dato,Valores Únicos,Total Valores,Valores Nulos,Porcentaje Nulos
0,PropertyId,int64,223,38893,0,0.000000
1,CalendarDate,datetime64[ns],184,38893,0,0.000000
2,Blocked,int64,3,38893,0,0.000000
3,Property_BookingId,float64,6160,31168,7725,19.862186
4,MaxCalendarDate,datetime64[ns],3,38893,0,0.000000
5,MinCalendarDate,datetime64[ns],21,38893,0,0.000000
6,DiasDiferencia,int64,22,38893,0,0.000000


Puedes encontrar el código en formato de cuaderno Jupyter (IPython Notebook) en el siguiente enlace: https://github.com/davidsosaolea/Stay-Unique.